In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from crowd_evaluation import ConfidenceEvaluatorC
from datasets import SyntheticDataset
from tqdm.notebook import tqdm
from joblib import Memory, Parallel, delayed


memory = memory = Memory('density_vs_interval_size', verbose=0)
%matplotlib inline

In [3]:
confidence = 0.8
# configurations = [(300, 3)]
configurations = [(100,7), (300,3), (300,7)]
densities = np.arange(0.5, 1.0, 0.05)
num_iterations = 500

In [ ]:
@memory.cache
def calc_confidence_interval_size(num_tasks: int, num_workers: int, confidence: float, density: float, i: int):
    dataset = SyntheticDataset(num_samples=num_samples, num_workers=num_workers, sample_fractions=[density]*num_workers)
    evaluater = ConfidenceEvaluatorC(dataset)
    ps, cs = evaluater.evaluate_workers(dataset.workers, confidence=confidence)
    return np.average(cs)

results = dict()
error_count = 0
for num_samples, num_workers in configurations:
    results[(num_samples, num_workers)] = np.zeros(len(densities))
    print('\nCalculating for', num_samples, 'samples and', num_workers, 'workers.')
    for i, density in enumerate(densities):
        print('\tdensity', density)
        average_confidence = 0

        res = None
        while res is None:
            try:
                res = Parallel(n_jobs=6) \
                    (delayed(calc_confidence_interval_size)(num_samples, num_workers, confidence, density, j)
                 for j in range(num_iterations))
            except:
                print('Error occured')
                error_count += 1

        res = np.array(res)
        average_confidence_interval_size = np.sum(res) / num_iterations
        print('\t\tResult:', average_confidence_interval_size)
        results[(num_samples, num_workers)][i] = average_confidence_interval_size

print('Error count', error_count)
        # for j in range(num_iterations):
        #     pbar.update(1)
        #     dataset = SyntheticDataset(num_samples=num_samples, num_workers=num_workers, sample_fractions=[density]*num_workers)
        #     evaluater = EvaluaterC(dataset)
        #     ps, cs = evaluater.evaluate_workers(dataset.workers, confidence=confidence)
        #     average_confidence += np.average(cs)
        # average_confidence /= num_iterations


Calculating for 100 samples and 7 workers.
	density 0.5
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error occured
Error

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 175, in _get_module_lock
KeyError: 'nt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/opt/homebrew/lib/python3.9/site-packages/joblib/__init__.py", line 113, in <module>
    from .memory import Memory, MemorizedResult, register_store_backend
  File "/opt/homebrew/lib/python3.9/site-packages/joblib/memory.py", line 15, in <module>
    import pathlib
  File "/opt/homebrew/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/pathlib.py", 

In [1]:
mpl.rcParams['figure.dpi'] = 300

for num_samples, num_workers in configurations:
    plt.plot(densities, results[(num_samples, num_workers)], marker='s', linestyle='--',
                label =str(num_workers)+' workers, '+str(num_samples)+' tasks', linewidth=1, markersize=2)

plt.xlabel('Density')
plt.ylabel('Size of interval')
plt.legend()
plt.show()
plt.savefig('density_vs_interval_size.png')

NameError: name 'mpl' is not defined